# Pipeline 

In [2]:
%load_ext autoreload
%autoreload 2

from averitec import Datapoint
from evidence_generation import GptEvidenceGenerator, GptBatchedEvidenceGenerator, DynamicFewShotBatchedEvidenceGenerator
from classification import DefaultClassifier, HuggingfaceClassifier, AverageEnsembleClassifier, LogRegEnsembleClassifier
from retrieval import SimpleFaissRetriever, Retriever, MmrFaissRetriever, SubqueryRetriever
from pipeline import Pipeline, MockPipeline
import pickle
from labels import label2id, id2label
import numpy as np
from sklearn.metrics import classification_report
import random
from tqdm import tqdm
random.seed(111)

import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# use torch and print name of 0-th GPU
import torch
print(torch.cuda.get_device_name(0))

Tesla V100-SXM2-32GB


In [35]:
# go through /mnt/data/factcheck/averitec-data/data_store/batch_jobs_llama_res/dev_mmr+gpt4o-dfewshot-tiebrk-atype/batch_{i}_results.jsonl for i 1-5
# and get the results
from collections import defaultdict
import os
import json

results = []
for i in range(7, 8):
    with open(
        f"/mnt/data/factcheck/averitec-data/data_store/batch_jobs_llama_res/dev_mmr+gpt4o-dfewshot-tiebrk-atype/batch_{i}_results.jsonl",
        "r",
    ) as f:
        for line in f:
            results.append(json.loads(line))
            print(json.loads(line)["custom_id"])

with open(
    "/mnt/data/factcheck/averitec-data/data_store/batch_jobs/dev_mmr+llama-dfewshot-tiebrk-atype/output.jsonl",
    "w",
) as f:
    for result in results:
        f.write(
            json.dumps(
                {
                    "id": "batch_req_ABAWJ4IHetlz3CT7Pv1FCIZy",
                    "custom_id": result["custom_id"],
                    "response": {
                        "status_code": 200,
                        "request_id": "ccd9c82c92598c9b54348371528846db",
                        "body": {
                            "id": "chatcmpl-9odDOZVvLBIqmCrn1IwuDiIZlDu5R",
                            "object": "chat.completion",
                            "created": 1721854170,
                            "model": "gpt-4o-2024-05-13",
                            "choices": [
                                {
                                    "index": 0,
                                    "message": {"role": "assistant", "content": result["response"]},
                                    "logprobs": None,
                                    "finish_reason": "stop",
                                }
                            ],
                            "usage": {"prompt_tokens": 5066, "completion_tokens": 611, "total_tokens": 5677},
                            "system_fingerprint": "fp_400f27fa1f",
                        },
                    },
                    "error": None,
                }
            )
            + "\n"
        )

averitec-0
averitec-1
averitec-2
averitec-3
averitec-4
averitec-5
averitec-6
averitec-7
averitec-8
averitec-9
averitec-10
averitec-11
averitec-12
averitec-13
averitec-14
averitec-15
averitec-16
averitec-17
averitec-18
averitec-19
averitec-20
averitec-21
averitec-22
averitec-23
averitec-24
averitec-25
averitec-26
averitec-27
averitec-28
averitec-29
averitec-30
averitec-31
averitec-32
averitec-33
averitec-34
averitec-35
averitec-36
averitec-37
averitec-38
averitec-39
averitec-40
averitec-41
averitec-42
averitec-43
averitec-44
averitec-45
averitec-46
averitec-47
averitec-48
averitec-49
averitec-50
averitec-51
averitec-52
averitec-53
averitec-54
averitec-55
averitec-56
averitec-57
averitec-58
averitec-59
averitec-60
averitec-61
averitec-62
averitec-63
averitec-64
averitec-65
averitec-66
averitec-67
averitec-68
averitec-69
averitec-70
averitec-71
averitec-72
averitec-73
averitec-74
averitec-75
averitec-76
averitec-77
averitec-78
averitec-79
averitec-80
averitec-81
averitec-82
averitec-83
av

## Pipeline test

In [6]:
split = "dev"
path = "/mnt/data/factcheck/averitec-data/"
with open(path + f"data/{split}.json") as f:
    dataset = json.load(f)
    for i in range(len(dataset)):
        dataset[i]["claim_id"] = i
    datapoints = [Datapoint.from_dict(d) for d in dataset]
    

In [7]:
datapoint = Datapoint.from_dict(dataset[150])
datapoint

Datapoint(claim='Zimbabwe gets more than half of the African Export Import Bank loans.', claim_id=150, claim_date='7-10-2020', speaker='Tendai Biti', original_claim_url='https://web.archive.org/web/20210129100929im_/https://lh4.googleusercontent.com/WwmjzhYNC1g84X0JmQzruGRgW4tjrB8rAPT9Zx6x-W6xBhQpVIwGW1zw7wImZ6R6JAyQywo-CiCE3_AZtJcxB74aev7jVG1PFobnm3PKanSX0bwWP3Z8E6z-6tDwuPgrSa6mhuz7', reporting_source='Twitter', location_ISO_code='ZW', label='Refuted', metadata={'required_reannotation': False, 'justification': 'Seems the numbers is no where near half. Therefore the claim is refuted', 'fact_checking_article': 'https://web.archive.org/web/20210129100929/https://zimfact.org/does-zimbabwe-get-more-than-half-afreximbank-loans/', 'claim_types': ['Numerical Claim'], 'fact_checking_strategies': ['Numerical Comparison'], 'questions': [{'question': 'How much has Zimbabwe gotten from the Africa Export Import Bank', 'answers': [{'answer': 'Zimbabwe got US$2,78 billion.', 'answer_type': 'Abstracti

In [8]:
# retriever = SimpleFaissRetriever(path="/mnt/data/factcheck/averitec-data/data_store/vecstore/dev/6k")
retriever = MmrFaissRetriever(path=f"/mnt/data/factcheck/averitec-data/data_store/vecstore/{split}/6k")
#retriever = MmrFaissRetriever(path=f"/mnt/data/factcheck/averitec-data/data_store/vecstore/{split}/2k")
retrieval_result = retriever(datapoint)
retrieval_result

RetrievalResult(documents=[Document(metadata={'url': 'https://www.voanews.com/a/afreximbank-pledges-to-post-mugabe-zimbabwe/4161541.html', 'context_before': '', 'context_after': 'The southern African nation, which dumped its hyperinflation-hit currency in 2009, is struggling with a severe dollar crunch that has seen banks fail to avail cash to customers while importers struggle to pay for imports. Finance Minister Patrick Chinamasa promised in a budget speech last week to re-engage with international lenders, curb spending and attract investors to revive the economy. '}, page_content='The African Export and Import Bank has pledged up to $1.5 billion in new loans and financial guarantees to Zimbabwe in a major boost for new President Emmerson Mnangagwa\'s government, the bank\'s president and chairman said Tuesday. Mnangagwa, who took over last month after veteran autocrat Robert Mugabe quit following a de facto military coup, has vowed to focus on reviving the struggling economy and pr

In [10]:
datapoint = Datapoint.from_dict(dataset[2])
datapoint.claim, datapoint.claim_date, datapoint.speaker

("Due to Imran Khan's criticism of Macron's comments on Islam, French authorities cancelled the visas of 183 Pakistani citizens and deported 118 from the country.",
 '31-10-2020',
 'Consulate General Of Pakistan France')

In [ ]:
json.dumps({"question": datapoint.claim, "context": datapoint.speaker})

In [26]:
from classification import DefaultClassifier, HuggingfaceClassifier, AverageEnsembleClassifier, LogRegEnsembleClassifier, RandomForestClassifier, NoTiebreakClassifier

path = "/mnt/data/factcheck/averitec-data/"
#target = path + "data_store/vecstore/test/2k"
PIPELINE_NAME = "mmr+llama-dfewshot-tiebrk-atype"
classifier = DefaultClassifier()
if True:
    pipeline = MockPipeline(
        dumps=f"/mnt/data/factcheck/averitec-data/data_store/submissions/{split}_mmr+gpt4o-dfewshot-tiebrk-atype.pkl",
        evidence_generator=DynamicFewShotBatchedEvidenceGenerator(),
        classifier=NoTiebreakClassifier()
    )
else:
    pipeline = Pipeline(
        #dumps = "/mnt/data/factcheck/averitec-data/data_store/submissions/dev_mmr+gpt4o-dfewshot.pkl",
        #SubqueryRetriever(retriever),
        retriever,
        evidence_generator=DynamicFewShotBatchedEvidenceGenerator(), 
        classifier=classifier
    )

submission = []
dump = []

for dp in tqdm(datapoints):
    pipeline_result = pipeline(dp)
    submission.append(pipeline_result.to_submission())
    dump.append(pipeline_result)
with open(f"/mnt/data/factcheck/averitec-data/data_store/submissions/{split}_{PIPELINE_NAME}.json", "w") as f:
    json.dump(submission, f, indent=4)
with open(f"/mnt/data/factcheck/averitec-data/data_store/submissions/{split}_{PIPELINE_NAME}.pkl", "wb") as f:
    pickle.dump(dump, f)

100%|██████████| 500/500 [00:05<00:00, 96.50it/s] 


In [13]:
files = pipeline.evidence_generator.get_batch_files(path=f"/mnt/data/factcheck/averitec-data/data_store/batch_jobs/{split}_{PIPELINE_NAME}", batch_size=90)

In [14]:
files

['/mnt/data/factcheck/averitec-data/data_store/batch_jobs/dev_mmr+llama-dfewshot-tiebrk-atype/batch_1.jsonl',
 '/mnt/data/factcheck/averitec-data/data_store/batch_jobs/dev_mmr+llama-dfewshot-tiebrk-atype/batch_2.jsonl',
 '/mnt/data/factcheck/averitec-data/data_store/batch_jobs/dev_mmr+llama-dfewshot-tiebrk-atype/batch_3.jsonl',
 '/mnt/data/factcheck/averitec-data/data_store/batch_jobs/dev_mmr+llama-dfewshot-tiebrk-atype/batch_4.jsonl',
 '/mnt/data/factcheck/averitec-data/data_store/batch_jobs/dev_mmr+llama-dfewshot-tiebrk-atype/batch_5.jsonl',
 '/mnt/data/factcheck/averitec-data/data_store/batch_jobs/dev_mmr+llama-dfewshot-tiebrk-atype/batch_6.jsonl']

In [18]:
batch_results = pipeline.evidence_generator.submit_and_await_batches(files, f"/mnt/data/factcheck/averitec-data/data_store/batch_jobs/{split}_{PIPELINE_NAME}/output.jsonl")

!!!!! existing outfile found, skipping computation


In [36]:
# load /mnt/data/factcheck/averitec-data/data_store/batch_jobs/dev_mmr+gpt4o/output.jsonl 
batch_results = []
with open(f"/mnt/data/factcheck/averitec-data/data_store/batch_jobs/{split}_{PIPELINE_NAME}/output.jsonl", "r") as f:
    for line in f:
        batch_results.append(json.loads(line)["response"]["body"]["choices"][0]["message"]["content"])

In [37]:
new_dump = []
pipeline.evidence_generator.fallback_gpt_generator.client.temperature = .5
for pipeline_result, batch_result in zip(dump[:len(batch_results)], batch_results):
    
    new_result = pipeline.evidence_generator.update_pipeline_result(pipeline_result, batch_result, pipeline.classifier)
    new_dump.append(new_result)


Error parsing JSON for EvidenceGenerator.
 
{
    "questions":
        [
            {"question": "Do the Democrats want to ban fracking?", "answer": "Some Democrats want to ban fracking, but not all of them.", "source": "6", "answer_type":"Abstractive"},
            {"question": "What is fracking?", "answer": "Fracking is a process in which millions of gallons of water, sand and chemicals are injected into wells miles deep to break up layers of shale, releasing oil or gas trapped in layers of rock.", "source": "10", "answer_type":"Extractive"},
            {"question": "Do the Democrats have a unified stance on fracking?", "answer": "No, they do not. While some Democrats, such as Bernie Sanders and Elizabeth Warren, have called for a ban on fracking, others, such as Joe Biden, have not.", "source": "10", "answer_type":"Abstractive"},
            {"question": "What are the arguments in favor of banning fracking?", "answer": "Some environmentalists argue that fracking’s poor polling sho

In [38]:
(
    new_dump[1],
    new_dump[1].evidence_generation_result,
    new_dump[1].classification_result
)

(PipelineResult(datapoint=Datapoint(claim='Trump Administration claimed songwriter Billie Eilish Is Destroying Our Country In Leaked Documents', claim_id=1, claim_date='31-10-2020', speaker=None, original_claim_url=None, reporting_source='Instagram', location_ISO_code='US', label='Refuted', metadata={'required_reannotation': False, 'justification': 'Seems that the Wzshington post accused the singer wrong lyrics. Therefore the claim is refuted', 'fact_checking_article': 'https://web.archive.org/web/20201103001419/https://leadstories.com/hoax-alert/2020/11/fact-check-trump-administration-did-not-claim-songwriter-billie-eilish-was-destroying-country-in-leaked-documents.html', 'claim_types': ['Position Statement', 'Event/Property Claim'], 'fact_checking_strategies': ['Written Evidence'], 'questions': [{'question': 'Has the Trump administration voiced that Billie Eilish was destroying the country', 'answers': [{'answer': 'A Washington Post story wrongly claimed the Trump administration accu

In [32]:
new_dump[-1]

PipelineResult(datapoint=Datapoint(claim='The first night of the US Republican National Convention had 128.4 million viewers', claim_id=499, claim_date='26-8-2020', speaker='Absolute Patriot states', original_claim_url='https://www.facebook.com/absolutepatriot/posts/1592221964271840?__xts__[0]=68.ARAKkfteHFa0qXRHfbmW-xrfjf7XVffZPRWtFJrd92Dlt6HE0IknLK2UyoUCQzyYPWKyRkdMAtslLoY3zFQ_WJJC3rol4PXpf1-WLlHDm21sBCbQ7QKpWEt9wXqqkYT__vddGsYOgxzxHC9l19kkqdBQyiLrZknMP4CPyhVOy9giNPHj6RkicugmbzzBzH23iMUBj5rlG8oYDfRl-jiZidHja4-V2KAu8zSg0yepEu0u8DjRhC4oMqe4KakoCEY9y82DO5JUd2nMZclA5AX8DeL4sLrAMl9fn4cJjvKQHUjDWlDucjBmha7UtupcTdX5Jchvjiwm7FIQAr2xhwTWWg', reporting_source='Facebook', location_ISO_code='US', label='Refuted', metadata={'required_reannotation': True, 'justification': 'An estimated 17 million people tuned in to watch the first night of the 2020 Republican National Convention on Monday, August 24.', 'fact_checking_article': 'https://web.archive.org/web/20201101222911/https://www.usatoday.com/st

In [39]:
with open(f"/mnt/data/factcheck/averitec-data/data_store/submissions/{split}_{PIPELINE_NAME}.json", "w") as f:
    json.dump([d.to_submission() for d in new_dump], f, indent=4)
with open(f"/mnt/data/factcheck/averitec-data/data_store/submissions/{split}_{PIPELINE_NAME}.pkl", "wb") as f:
    pickle.dump(new_dump, f)

In [40]:
print(f"{split}_{PIPELINE_NAME}")
%run src/prediction/evaluate_veracity.py --label_file /mnt/data/factcheck/averitec-data/data/dev.json --prediction_file /mnt/data/factcheck/averitec-data/data_store/submissions/{split}_{PIPELINE_NAME}.json

dev_mmr+llama-dfewshot-tiebrk-atype
Question-only score (HU-meteor):             0.462721423131608
Question-answer score (HU-meteor):           0.2667748294276599
Veracity F1 scores:
 * Supported:                                0.7232472324723247
 * Refuted:                                  0.8298887122416534
 * Not Enough Evidence:                      0.18867924528301888
 * Conflicting Evidence/Cherrypicking:       0.0425531914893617
 * macro:                                    0.44609209537158967
 * acc:                                      0.73
--------------------
AVeriTeC scores:
 * Veracity scores (meteor @ 0.1):           0.7
 * Veracity scores (meteor @ 0.2):           0.502
 * Veracity scores (meteor @ 0.25):          0.356
 * Veracity scores (meteor @ 0.3):           0.246
 * Veracity scores (meteor @ 0.4):           0.096
 * Veracity scores (meteor @ 0.5):           0.032
--------------------
AVeriTeC scores by type @ 0.25:
 * Veracity scores (Event/Property Claim):   0.175

In [32]:
print(f"{split}_{PIPELINE_NAME}")
%run src/prediction/evaluate_veracity.py --label_file /mnt/data/factcheck/averitec-data/data/dev.json --prediction_file /mnt/data/factcheck/averitec-data/data_store/submissions/{split}_{PIPELINE_NAME}.json

dev_mmr+gpt4o-dfewshot-atype
Question-only score (HU-meteor):             0.4566753100185953
Question-answer score (HU-meteor):           0.28764311409757226
Veracity F1 scores:
 * Supported:                                0.7252747252747253
 * Refuted:                                  0.8217821782178217
 * Not Enough Evidence:                      0.23684210526315788
 * Conflicting Evidence/Cherrypicking:       0.044444444444444446
 * macro:                                    0.45708586330003736
 * acc:                                      0.716
--------------------
AVeriTeC scores:
 * Veracity scores (meteor @ 0.1):           0.696
 * Veracity scores (meteor @ 0.2):           0.546
 * Veracity scores (meteor @ 0.25):          0.418
 * Veracity scores (meteor @ 0.3):           0.3
 * Veracity scores (meteor @ 0.4):           0.136
 * Veracity scores (meteor @ 0.5):           0.052
--------------------
AVeriTeC scores by type @ 0.25:
 * Veracity scores (Event/Property Claim):   0.20766

In [7]:
print(f"{split}_{PIPELINE_NAME}")
%run src/prediction/evaluate_veracity.py --label_file /mnt/data/factcheck/averitec-data/data/dev.json --prediction_file /mnt/data/factcheck/averitec-data/data_store/submissions/{split}_{PIPELINE_NAME}.json

dev_mmr+gpt4o-dfewshot-tiebrk-atype
Question-only score (HU-meteor):             0.4566753100185953
Question-answer score (HU-meteor):           0.2878181626887146
Veracity F1 scores:
 * Supported:                                0.7368421052631579
 * Refuted:                                  0.813953488372093
 * Not Enough Evidence:                      0.2222222222222222
 * Conflicting Evidence/Cherrypicking:       0.0784313725490196
 * macro:                                    0.46286229710162313
 * acc:                                      0.708
--------------------
AVeriTeC scores:
 * Veracity scores (meteor @ 0.1):           0.688
 * Veracity scores (meteor @ 0.2):           0.54
 * Veracity scores (meteor @ 0.25):          0.414
 * Veracity scores (meteor @ 0.3):           0.292
 * Veracity scores (meteor @ 0.4):           0.132
 * Veracity scores (meteor @ 0.5):           0.052
--------------------
AVeriTeC scores by type @ 0.25:
 * Veracity scores (Event/Property Claim):   0.20

## collapsible begin

In [ ]:
from IPython.display import display, Markdown, Latex

In [ ]:
knn_retrieval_result = retriever(datapoint)
display(Markdown("### 🗯️ " + datapoint.claim))
display(Markdown("*Retrieved by knn*\n\n"))
# sample 3
for r in knn_retrieval_result:
    newline = "\n"
    display(Markdown(f"**{r.metadata['url']}**\n\n{r.page_content[:256]}"))

In [ ]:
from retrieval import MmrFaissRetriever

mmr_retriever = MmrFaissRetriever(retriever.path)
mmr_retrieval_result = mmr_retriever(datapoint)
display(Markdown("### 🗯️ " + datapoint.claim))
display(Markdown("*Retrieved by MMR*\n\n"))
# sample 3
for r in mmr_retrieval_result:
    newline = "\n"
    display(Markdown(f"**{r.metadata['url']}**\n\n{r.page_content[:256]}"))

In [ ]:
subquery_retriever = SubqueryRetriever(retriever)
subquery_retrieval_result = subquery_retriever(datapoint)
display(Markdown("### 🗯️ " + datapoint.claim))
display(Markdown("*Retrieved by subqueries*\n\n"))
# sample 3
for r in subquery_retrieval_result:
    newline = "\n"
    display(Markdown(f"**{r.metadata['url']}**\n\n*{';'.join(r.metadata['queries'])}*\n\n{r.page_content[:256]}"))

In [ ]:
subquery_retrieval_result.metadata

## Collapsible section end

In [ ]:
evidence_generator = GptBatchedEvidenceGenerator("gpt-4o")
evidence_generation_result = evidence_generator(datapoint, retrieval_result)
evidence_generation_result

In [ ]:
evidence_generation_result.metadata["suggested_label"]

In [ ]:
datapoint.label

In [ ]:
classifier = DefaultClassifier()
classification_result = classifier(datapoint, evidence_generation_result, retrieval_result)
str(classification_result), classification_result

In [ ]:
datapoint2 = Datapoint.from_dict(dataset[16])
pipeline = Pipeline(retriever, evidence_generator, classifier)
pipeline_result = pipeline(datapoint2)
pipeline_result

In [ ]:
str(pipeline_result.classification_result), datapoint2.label

In [ ]:
pipeline_result.to_submission()

In [ ]:
# pickle dump pipeline result
import pickle
with open('data/pipeline_result.pkl', 'wb') as f:
    pickle.dump(pipeline_result, f)

In [ ]:
%run src/prediction/evaluate_veracity.py --label_file /mnt/data/factcheck/averitec-data/data/dev.json --prediction_file /mnt/data/factcheck/averitec-data/data_store/submission_dev_avg_clf.json